In [11]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import Subset
import torch.optim as optim
import torch.nn.functional as F


In [12]:
# === CONFIG ===
INPUT_DIR = "/home/HardDisk/Satang/thesis_proj/New_45/15/split_tws/X_csv_split_31"
CHUNK_SIZE = 31
NUM_FEATURES = 8
NUM_EPOCHS = 50
BATCH_SIZE = 32
K_FOLDS = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
# === HELPERS ===
def collate_fn(batch):
    streams, labels = zip(*batch)
    streams = list(zip(*streams))
    streams = [torch.stack(s) for s in streams]
    labels = torch.tensor(labels)
    return streams, labels

def compute_class_weights(labels, device):
    counter = Counter(labels)
    total = sum(counter.values())
    weights = [np.log(total / (counter[i] + 1)) for i in range(len(counter))]
    return torch.tensor(weights, dtype=torch.float).to(device)

def print_model_info(model):
    print("Model Architecture:")
    print(model)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")

def plot_loss_curve(train_losses, val_losses):
    epochs = range(len(train_losses))
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_losses, label='Train Loss', color='blue')
    plt.plot(epochs, val_losses, label='Val Loss', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_accuracy_curve(train_acc, val_acc):
    epochs = range(len(train_acc))
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_acc, label='Train Accuracy', color='green')
    plt.plot(epochs, val_acc, label='Val Accuracy', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [14]:
# === DATASET CLASS ===
class MultiStreamDataset(Dataset):
    def __init__(self, data, labels, label_encoder, augment=False):
        self.data = data
        self.labels = label_encoder.transform(labels)
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def augment_stream(self, stream):
        jitter = np.random.normal(0, 0.01, stream.shape)
        scale = np.random.normal(1.0, 0.05, stream.shape)
        return stream * scale + jitter

    def __getitem__(self, idx):
        sample = self.data[idx]  # shape: (T, 8)
        
        if self.augment:
            # Apply augmentation to each feature independently
            jitter = np.random.normal(0, 0.01, sample.shape)
            scale = np.random.normal(1.0, 0.05, sample.shape)
            sample = sample * scale + jitter

        sample_tensor = torch.tensor(sample, dtype=torch.float32)  # shape: (T, 8)
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.long)
    
        return sample_tensor, label_tensor


In [15]:
class DNNTeacher(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(DNNTeacher, self).__init__()
        T, F = input_shape
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(T * F, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.out = nn.Linear(128, num_classes)
        self.proj = nn.Linear(128, 128)  # ✅ matches saved model

    def forward(self, x, return_features=False):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        feat = F.relu(self.fc3(x))
        logits = self.out(feat)
        if return_features:
            return logits, self.proj(feat)
        return logits


In [16]:

# # === COLLATE FUNCTION ===
# def collate_fn(batch):
#     streams, labels = zip(*batch)
#     streams = list(zip(*streams))
#     streams = [torch.stack(s) for s in streams]
#     labels = torch.tensor(labels)
#     return streams, labels

# === LOAD SPLIT FUNCTION ===
def load_split_from_folder(split_dir, expected_shape):
    X, y = [], []
    for class_name in sorted(os.listdir(split_dir)):
        class_path = os.path.join(split_dir, class_name)
        for fname in sorted(os.listdir(class_path)):
            if fname.endswith(".csv"):
                fpath = os.path.join(class_path, fname)
                chunk = pd.read_csv(fpath, header=None).values
                if chunk.shape == expected_shape:
                    X.append(chunk)
                    y.append(class_name)
    return np.array(X), np.array(y)

In [17]:
# === APPLY SMOTE ===
def apply_smote_on_training(X_chunks, y_labels, chunk_size, num_features):
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_labels)

    smote_encoder = LabelEncoder()
    y_encoded_for_smote = smote_encoder.fit_transform(y_labels)

    X_flat = X_chunks.reshape(X_chunks.shape[0], -1)
    X_resampled, y_resampled = SMOTE().fit_resample(X_flat, y_encoded_for_smote)

    X_res = X_resampled.reshape(-1, chunk_size, num_features)
    y_res_str = smote_encoder.inverse_transform(y_resampled)

    return X_res, y_res_str, label_encoder



In [18]:
# === CLASS WEIGHTING ===
def compute_class_weights(labels, device):
    from collections import Counter
    total = len(labels)
    counts = Counter(labels)
    weights = [np.log(total / (counts[i] + 1)) for i in range(len(counts))]
    return torch.tensor(weights, dtype=torch.float).to(device)

# class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device="cpu")

In [19]:
def train_model(model, train_loader, val_loader, device, epochs=50, lr=0.001,
                  class_weights=None, optimizer=None, scheduler=None,
                  best_model_path="mlpmixer_teacher.pth"):

    print("🔍 Class Weights Tensor:")
    print(class_weights)

    criterion_ce = nn.CrossEntropyLoss(weight=class_weights)

    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_acc = 0

    for epoch in range(epochs):  # ✅ Outer loop: EPOCHS
        model.train()
        total_loss, correct = 0.0, 0

        if scheduler:
            scheduler.step()

        for inputs, labels in train_loader:  # ✅ ← this must be indented
            inputs, labels = inputs.to(device), labels.to(device)
            logits = model(inputs)

            loss = criterion_ce(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()

            total_loss += loss.item()
            correct += (logits.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)

        # === Validation ===
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logits = model(inputs)

                loss = criterion_ce(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {total_loss:.4f} - Val Loss: {val_loss:.4f} - "
              f"Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"💾 Best model saved to: {best_model_path}")

    model.load_state_dict(torch.load(best_model_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [20]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE

# === 🧩 Config ===
NUM_EPOCHS = 50
BEST_LR = 0.01
NUM_FEATURES = 8  # Assuming you use 8D input
BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_dir = "/home/HardDisk/Satang/thesis_proj"
save_root = "/home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher"
os.makedirs(save_root, exist_ok=True)

detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]

for T_d in detection_times:
    for T_w in window_sizes:
        print(f"\n🚀 Running for Td={T_d}, Tw={T_w}")
        model_name = os.path.join(save_root, f"dnn_Td{T_d}_Tw{T_w}.pth")
        T_len = T_d - T_w + 1
        folder_name = f"X_csv_split_{T_len}"

        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")

        # === 1. Load Data ===
        expected_shape = (T_len, NUM_FEATURES)
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. SMOTE + Encode ===
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)
        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, expected_shape[0], NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Datasets + Loaders ===
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === 4. MLP-Mixer Model ===
        model = DNNTeacher(
            input_shape=expected_shape,  # (T_len, NUM_FEATURES)
            num_classes=len(label_encoder.classes_)
        ).to(device)

        optimizer = optim.AdamW(model.parameters(), lr=BEST_LR, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 5. Train MLP-Mixer ===
        train_accs, val_accs, train_losses, val_losses = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            class_weights=class_weights_tensor,
            lr=BEST_LR,
            optimizer=optimizer,
            scheduler=scheduler,
            best_model_path=model_name
        )

        print(f"✅ Finished Td={T_d}, Tw={T_w} — saved to {model_name}")



🚀 Running for Td=30, Tw=10


🔍 Class Weights Tensor:
tensor([2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843,
        2.4843, 2.4843, 2.4843], device='cuda:0')


/home/ea301b/anaconda3/envs/dl_satang/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1/50 - Train Loss: 289.8578 - Val Loss: 32.8921 - Train Acc: 0.6045 - Val Acc: 0.5797
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td30_Tw10.pth
Epoch 2/50 - Train Loss: 205.8933 - Val Loss: 30.0223 - Train Acc: 0.7237 - Val Acc: 0.6047
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td30_Tw10.pth
Epoch 3/50 - Train Loss: 162.8922 - Val Loss: 23.7790 - Train Acc: 0.7806 - Val Acc: 0.6884
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td30_Tw10.pth
Epoch 4/50 - Train Loss: 154.4748 - Val Loss: 22.1381 - Train Acc: 0.7954 - Val Acc: 0.7089
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td30_Tw10.pth
Epoch 5/50 - Train Loss: 137.1254 - Val Loss: 19.5697 - Train Acc: 0.8204 - Val Acc: 0.7350
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/t

/tmp/ipykernel_2425667/2772929862.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


...

/tmp/ipykernel_2425667/2772929862.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


🔍 Class Weights Tensor:
tensor([2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835,
        2.4835, 2.4835, 2.4835], device='cuda:0')


/home/ea301b/anaconda3/envs/dl_satang/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1/50 - Train Loss: 149.6178 - Val Loss: 11.0472 - Train Acc: 0.6025 - Val Acc: 0.6557
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td60_Tw20.pth
Epoch 2/50 - Train Loss: 69.7754 - Val Loss: 10.6711 - Train Acc: 0.8107 - Val Acc: 0.7002
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td60_Tw20.pth
Epoch 3/50 - Train Loss: 61.2312 - Val Loss: 7.8660 - Train Acc: 0.8339 - Val Acc: 0.7834
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td60_Tw20.pth
Epoch 4/50 - Train Loss: 45.7261 - Val Loss: 10.5800 - Train Acc: 0.8752 - Val Acc: 0.7717
Epoch 5/50 - Train Loss: 42.1314 - Val Loss: 8.6342 - Train Acc: 0.8833 - Val Acc: 0.7646
Epoch 6/50 - Train Loss: 40.2813 - Val Loss: 7.1285 - Train Acc: 0.8944 - Val Acc: 0.8068
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/teacher/dnn_Td60_Tw20.pth
Epoc

/tmp/ipykernel_2425667/2772929862.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


In [21]:
import torch
import torch.nn as nn

class StudentCNN(nn.Module):
    def __init__(self, input_length, num_classes=12):
        super().__init__()
        self.streams = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(1, 4, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv1d(4, 8, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveAvgPool1d(1),
                nn.Flatten()
            ) for _ in range(8)
        ])

        self.fc = nn.Sequential(
            nn.Linear(8 * 8, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

        self.proj = nn.Linear(8 * 8, 128)

    def forward(self, x, return_features=False):
        B, T, C = x.shape
        assert C == 8, f"Expected 8 feature streams, got {C}"

        # Process each feature stream independently
        streams = [x[:, :, i].unsqueeze(1) for i in range(C)]
        features = [self.streams[i](streams[i]) for i in range(8)]

        x = torch.cat(features, dim=1)  # (B, 64)
        logits = self.fc(x)

        if return_features:
            feat_proj = self.proj(x)
            return logits, feat_proj
        return logits


In [22]:
import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, student_feat, teacher_feat, true_labels, T=3.0, alpha=0.5, beta=0.3, gamma=0.2):
    """
    alpha: weight for hard loss (CE)
    beta: weight for soft loss (KL)
    gamma: weight for feature-based distillation (MSE)
    T: temperature for soft distillation
    """
    # Hard loss
    ce_loss = F.cross_entropy(student_logits, true_labels)

    # Soft loss (logits)
    soft_teacher = F.softmax(teacher_logits / T, dim=1)
    soft_student = F.log_softmax(student_logits / T, dim=1)
    kl_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (T * T)

    # Feature loss (projection-matched MSE)
    feat_loss = F.mse_loss(student_feat, teacher_feat)

    return alpha * ce_loss + beta * kl_loss + gamma * feat_loss


In [23]:
def train_distilled(student, teacher, train_loader, val_loader, device,
                    epochs=50, lr=0.0005, class_weights=None,
                    T=3.0, alpha=0.5, beta=0.3, gamma=0.2,
                    save_path="best_student.pth"):

    teacher.eval()
    student.to(device)
    teacher.to(device)

    optimizer = torch.optim.Adam(student.parameters(), lr=lr)
    ce_criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_val_acc = 0
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        student.train()
        total_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # === Forward passes (must pass return_features=True)
            student_logits, student_feat = student(inputs, return_features=True)
            with torch.no_grad():
                teacher_logits, teacher_feat = teacher(inputs, return_features=True)

            # === KD loss
            loss = distillation_loss(
                student_logits, teacher_logits,
                student_feat, teacher_feat,
                labels,
                T=T, alpha=alpha, beta=beta, gamma=gamma
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (student_logits.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)

        # === Validation (logits only)
        student.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                logits = student(inputs)  # ← returns only logits
                loss = ce_criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"[Distill] Epoch {epoch+1:02d}/{epochs} - "
              f"Loss: {total_loss:.4f} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student.state_dict(), save_path)
            print(f"💾 Best student model saved to: {save_path}")

    student.load_state_dict(torch.load(save_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [32]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import os
import torch
from torch.utils.data import DataLoader

# === Configs ===
NUM_FEATURES = 8
BATCH_SIZE = 64
NUM_EPOCHS = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]

base_dir = "/home/HardDisk/Satang/thesis_proj"
teacher_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","dnn", "teacher")  # ← not AE
student_save_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","dnn", "student")
os.makedirs(student_save_dir, exist_ok=True)

for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)

        print(f"\n🚀 Distilling DNN → StudentCNN for Td={T_d}, Tw={T_w} (T_len={T_len})")

        # === Paths
        folder_name = f"X_csv_split_{T_len}"
        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")
        teacher_path = os.path.join(teacher_dir, f"dnn_Td{T_d}_Tw{T_w}.pth")
        student_path = os.path.join(student_save_dir, f"student_Td{T_d}_Tw{T_w}.pth")

        # === 1. Load Data
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. Encode & Balance
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)
        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, T_len, NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Dataset & Dataloader
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 4. Load MLP-Mixer Teacher
        teacher = DNNTeacher(
            input_shape=(T_len, NUM_FEATURES),
            num_classes=len(label_encoder.classes_)
        )
        teacher.load_state_dict(torch.load(teacher_path, map_location=device))
        teacher.to(device)
        teacher.eval()
        for p in teacher.parameters():
            p.requires_grad = False

        # === 5. Init StudentCNN
        student = StudentCNN(
            input_length=T_len,
            num_classes=len(label_encoder.classes_)
        ).to(device)

        # === 6. Train with Knowledge Distillation
        train_accs, val_accs, train_losses, val_losses = train_distilled(
            student=student,
            teacher=teacher,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            lr=0.001,
            class_weights=class_weights_tensor,
            T=2.0,
            alpha=0.9,
            beta=0.5,
            gamma=0.1,
            save_path=student_path
        )

        # === 7. Final Evaluation
        student.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                # ✅ Only get logits for prediction
                logits = student(inputs)  # <- returns only logits when return_features=False
                preds = torch.argmax(logits, dim=1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print("\n📊 Final Classification Report (StudentCNN):")
        print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))
        print(f"✅ Student model saved: {student_path}")



🚀 Distilling DNN → StudentCNN for Td=30, Tw=10 (T_len=21)


/tmp/ipykernel_2425667/1599356996.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load(teacher_path, map_location=device))


[Distill] Epoch 01/50 - Loss: 1159.3909 - Train Acc: 0.5220 - Val Acc: 0.5912
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw10.pth
[Distill] Epoch 02/50 - Loss: 620.0776 - Train Acc: 0.7205 - Val Acc: 0.6237
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw10.pth
[Distill] Epoch 03/50 - Loss: 494.4442 - Train Acc: 0.7723 - Val Acc: 0.7370
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw10.pth
[Distill] Epoch 04/50 - Loss: 404.8286 - Train Acc: 0.8096 - Val Acc: 0.7710
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw10.pth
[Distill] Epoch 05/50 - Loss: 342.8343 - Train Acc: 0.8384 - Val Acc: 0.8181
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td3

/tmp/ipykernel_2425667/1062020587.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))
/tmp/ipykernel_2425667/1599356996.py:62

[Distill] Epoch 01/50 - Loss: 3274.7697 - Train Acc: 0.5918 - Val Acc: 0.6374
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw15.pth
[Distill] Epoch 02/50 - Loss: 2498.8267 - Train Acc: 0.7480 - Val Acc: 0.6929
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw15.pth
[Distill] Epoch 03/50 - Loss: 2086.5321 - Train Acc: 0.7908 - Val Acc: 0.7244
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw15.pth
[Distill] Epoch 04/50 - Loss: 1677.9780 - Train Acc: 0.8123 - Val Acc: 0.7563
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td30_Tw15.pth
[Distill] Epoch 05/50 - Loss: 1427.8651 - Train Acc: 0.8268 - Val Acc: 0.7715
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student

...

/tmp/ipykernel_2425667/1062020587.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))
/tmp/ipykernel_2425667/1599356996.py:62

[Distill] Epoch 01/50 - Loss: 980.0618 - Train Acc: 0.4551 - Val Acc: 0.5873
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw15.pth
[Distill] Epoch 02/50 - Loss: 592.7374 - Train Acc: 0.7796 - Val Acc: 0.6785
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw15.pth
[Distill] Epoch 03/50 - Loss: 486.2953 - Train Acc: 0.8330 - Val Acc: 0.7203
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw15.pth
[Distill] Epoch 04/50 - Loss: 435.8782 - Train Acc: 0.8568 - Val Acc: 0.7646
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw15.pth
[Distill] Epoch 05/50 - Loss: 400.3995 - Train Acc: 0.8784 - Val Acc: 0.7759
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60

/tmp/ipykernel_2425667/1062020587.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))
/tmp/ipykernel_2425667/1599356996.py:62

[Distill] Epoch 01/50 - Loss: 893.3659 - Train Acc: 0.3841 - Val Acc: 0.4590
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw20.pth
[Distill] Epoch 02/50 - Loss: 551.0809 - Train Acc: 0.6834 - Val Acc: 0.5738
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw20.pth
[Distill] Epoch 03/50 - Loss: 418.8647 - Train Acc: 0.7689 - Val Acc: 0.6241
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw20.pth
[Distill] Epoch 04/50 - Loss: 358.6380 - Train Acc: 0.8060 - Val Acc: 0.6850
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60_Tw20.pth
[Distill] Epoch 05/50 - Loss: 324.7125 - Train Acc: 0.8255 - Val Acc: 0.7646
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/student/student_Td60

/tmp/ipykernel_2425667/1062020587.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_student_baseline(student, train_loader, val_loader, device,
                           epochs=50, lr=0.0005, class_weights=None,
                           save_path="best_student_baseline.pth"):

    student.to(device)
    optimizer = optim.Adam(student.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
        student.train()
        running_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 🔧 Always call with return_features=True for consistency
            logits, _ = student(inputs, return_features=True)
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct += (logits.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / len(train_loader.dataset)
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        # === Validation ===
        student.eval()
        val_running_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logits, _ = student(inputs, return_features=True)
                loss = criterion(logits, labels)

                val_running_loss += loss.item() * inputs.size(0)
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_loss = val_running_loss / len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"[Baseline] Epoch {epoch+1:02d}/{epochs} - "
              f"Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - "
              f"Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student.state_dict(), save_path)
            print(f"💾 Best standalone student model saved to: {save_path}")

    student.load_state_dict(torch.load(save_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [34]:
import os
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader

baseline_save_dir = "/home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student"
os.makedirs(baseline_save_dir, exist_ok=True)

for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)

        print(f"\n🚀 Training Standalone Student for Td={T_d}, Tw={T_w} (T_len={T_len})")

        # === Paths
        folder_name = f"X_csv_split_{T_len}"
        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")
        student_model_path = os.path.join(baseline_save_dir, f"student_baseline_Td{T_d}_Tw{T_w}.pth")

        # === 1. Load Raw Data
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. Encode Labels & Apply SMOTE
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)

        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, T_len, NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Dataset & DataLoader
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === 4. Class Weights
        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 5. Initialize StudentCNN
        student_baseline = StudentCNN(
            input_length=T_len,
            num_classes=len(label_encoder.classes_)
        ).to(device)

        # === 6. Train Student Model (No KD)
        train_accs_b, val_accs_b, train_losses_b, val_losses_b = train_student_baseline(
            student=student_baseline,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            lr=0.0001,
            class_weights=class_weights_tensor,
            save_path=student_model_path
        )

        print(f"✅ Saved standalone student model: {student_model_path}")



🚀 Training Standalone Student for Td=30, Tw=10 (T_len=21)


[Baseline] Epoch 01/50 - Train Loss: 2.4070 - Val Loss: 2.2428 - Train Acc: 0.1756 - Val Acc: 0.2685
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 02/50 - Train Loss: 1.6945 - Val Loss: 1.6298 - Train Acc: 0.4537 - Val Acc: 0.4599
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 03/50 - Train Loss: 1.3068 - Val Loss: 1.4049 - Train Acc: 0.5911 - Val Acc: 0.4955
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 04/50 - Train Loss: 1.1389 - Val Loss: 1.2636 - Train Acc: 0.6321 - Val Acc: 0.5506
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 05/50 - Tra

/tmp/ipykernel_2425667/486445689.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Train Loss: 2.2276 - Val Loss: 1.8238 - Train Acc: 0.2650 - Val Acc: 0.4210
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 02/50 - Train Loss: 1.3492 - Val Loss: 1.3808 - Train Acc: 0.5974 - Val Acc: 0.5501
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 03/50 - Train Loss: 1.0655 - Val Loss: 1.1968 - Train Acc: 0.6711 - Val Acc: 0.5961
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 04/50 - Train Loss: 0.9227 - Val Loss: 1.0742 - Train Acc: 0.7056 - Val Acc: 0.6317
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 05/50 - Tra

/tmp/ipykernel_2425667/486445689.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


...

/tmp/ipykernel_2425667/486445689.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Train Loss: 2.4979 - Val Loss: 2.4565 - Train Acc: 0.1018 - Val Acc: 0.1684
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 02/50 - Train Loss: 2.4079 - Val Loss: 2.3383 - Train Acc: 0.1838 - Val Acc: 0.3443
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 03/50 - Train Loss: 2.1320 - Val Loss: 1.9653 - Train Acc: 0.3687 - Val Acc: 0.4089
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 04/50 - Train Loss: 1.5566 - Val Loss: 1.5468 - Train Acc: 0.5584 - Val Acc: 0.5241
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 05/50 - Tra

/tmp/ipykernel_2425667/486445689.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Train Loss: 2.4801 - Val Loss: 2.4414 - Train Acc: 0.1060 - Val Acc: 0.1487
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 02/50 - Train Loss: 2.3575 - Val Loss: 2.2538 - Train Acc: 0.2196 - Val Acc: 0.3244
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 03/50 - Train Loss: 1.8017 - Val Loss: 1.6672 - Train Acc: 0.4572 - Val Acc: 0.4262
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 04/50 - Train Loss: 1.3957 - Val Loss: 1.5041 - Train Acc: 0.5721 - Val Acc: 0.5094
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/dnn/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 05/50 - Tra

/tmp/ipykernel_2425667/486445689.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


In [ ]:
import os
import torch
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader

# === CONFIG ===
detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]
NUM_FEATURES = 8
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_dir = "/home/HardDisk/Satang/thesis_proj/"
base_teacher_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","dnn", "teacher")
student_kd_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","dnn", "student")
student_baseline_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","dnn", "baseline_student")
csv_output_path = os.path.join(base_dir, "Deep_Learning", "cross_archi","dnn", "results", "model_eval_summary.csv")
os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)

results = []

def evaluate_and_log(model, model_path, test_loader, label_encoder, T_d, T_w, model_type, results_list):
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            # Handle models that return (logits, features)
            if hasattr(model, "forward") and "return_features" in model.forward.__code__.co_varnames:
                logits, _ = model(inputs, return_features=True)
            else:
                logits = model(inputs)

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    report = classification_report(all_labels, all_preds, target_names=label_encoder.classes_, output_dict=True)
    acc = accuracy_score(all_labels, all_preds)

    results_list.append({
        "Td": T_d,
        "Tw": T_w,
        "T_len": T_d - T_w + 1,
        "Model": model_type,
        "Accuracy": round(acc, 4),
        "Precision_macro": round(report["macro avg"]["precision"], 4),
        "Recall_macro": round(report["macro avg"]["recall"], 4),
        "F1_macro": round(report["macro avg"]["f1-score"], 4),
        "F1_weighted": round(report["weighted avg"]["f1-score"], 4),
    })

# === MAIN EVALUATION LOOP ===
for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)
        folder_name = f"X_csv_split_{T_len}"
        test_path = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name, "test")

        # === Load test set ===
        X_test_raw, y_test_raw = load_split_from_folder(test_path, expected_shape)
        label_encoder = LabelEncoder()
        label_encoder.fit(y_test_raw)

        test_dataset = MultiStreamDataset(X_test_raw, y_test_raw, label_encoder, augment=False)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === DNN Teacher ===
        teacher_path = os.path.join(base_teacher_dir, f"dnn_Td{T_d}_Tw{T_w}.pth")
        teacher = DNNTeacher(
            input_shape=(T_len, NUM_FEATURES),
            num_classes=len(label_encoder.classes_)
        )
        evaluate_and_log(teacher, teacher_path, test_loader, label_encoder, T_d, T_w, "Teacher", results)

        # === Student with KD ===
        student_kd_path = os.path.join(student_kd_dir, f"student_Td{T_d}_Tw{T_w}.pth")
        student_kd = StudentCNN(input_length=T_len, num_classes=len(label_encoder.classes_))
        evaluate_and_log(student_kd, student_kd_path, test_loader, label_encoder, T_d, T_w, "Student_KD", results)

        # === Student Baseline ===
        student_b_path = os.path.join(student_baseline_dir, f"student_baseline_Td{T_d}_Tw{T_w}.pth")
        student_b = StudentCNN(input_length=T_len, num_classes=len(label_encoder.classes_))
        evaluate_and_log(student_b, student_b_path, test_loader, label_encoder, T_d, T_w, "Student_Baseline", results)

# === Save to CSV ===
df_results = pd.DataFrame(results)
df_results.to_csv(csv_output_path, index=False)
print(f"\n✅ All evaluation results saved to {csv_output_path}")


/tmp/ipykernel_2425667/2486054416.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))
/tmp/ipykernel_2425


✅ All evaluation results saved to /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/dnn/results/model_eval_summary.csv


/tmp/ipykernel_2425667/2486054416.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))
